In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.preprocessing.image import ImageDataGenerator

print('Dependencies succesfully imported!')

"""Checking gpu device"""
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


Dependencies succesfully imported!
1 Physical GPUs, 1 Logical GPUs


In [2]:
# getting filepath for each split
train_path = r"dataset/train"
val_path = r"dataset/val"
test_path = r"dataset/test"

seed = 42
shuffle = True

datagen = ImageDataGenerator(
    rescale = 1/255.,
)

def augment_data(path, augmenter, batch_size = 32, target_size = (256,256), class_mode = 'binary'):
    return augmenter.flow_from_directory(
        directory = path,
        batch_size = batch_size,
        target_size = target_size,
        class_mode = class_mode,
        seed = seed,
        shuffle = shuffle
    )

# import files using image_dataset_from_directory
train = augment_data(train_path, augmenter = datagen)
val = augment_data(val_path, augmenter = datagen)
test = augment_data(test_path, augmenter = datagen)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [3]:
"""Creating CNNBlock"""
class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel_size, padding = 'same', input_shape = (None, None, None, None))
        self.batch_norm = layers.BatchNormalization()

    """Create function to run forward pass"""
    """CNN -> BatchNormalization -> ReLu"""
    def call(self, input, training = False):
        x = self.conv(input)
        print(x.shape)
        x = self.batch_norm(x, training = training)
        x = tf.nn.relu(x)
        return x

In [4]:
model = keras.Sequential([
    CNNBlock(32, 3),
    CNNBlock(64, 3),
    CNNBlock(128, 3),
    layers.Flatten(),
    layers.Dense(2)
])
model.compile(
    optimizer = keras.optimizers.Adam(learning_rate = 1e-4),
    loss = keras.losses.BinaryCrossentropy(),
    metrics = ['accuracy']
)

In [5]:
history = model.fit(
    train,
    batch_size = 64,
    epochs = 10,
)

(32, 256, 256, 32)
(32, 256, 256, 32)
Epoch 1/10
(None, None, None, 32)
(None, None, None, 32)
(None, None, None, 32)
163/163 [==============================] - 101s 612ms/step - loss: 3.9404 - accuracy: 0.7410
Epoch 2/10
163/163 [==============================] - 97s 593ms/step - loss: 3.9205 - accuracy: 0.7400
Epoch 3/10
163/163 [==============================] - 98s 598ms/step - loss: 3.9205 - accuracy: 0.7402
Epoch 4/10
163/163 [==============================] - 95s 581ms/step - loss: 3.9205 - accuracy: 0.7400
Epoch 5/10
163/163 [==============================] - 94s 579ms/step - loss: 3.9205 - accuracy: 0.7404
Epoch 6/10
163/163 [==============================] - 118s 724ms/step - loss: 3.9205 - accuracy: 0.7400
Epoch 7/10
163/163 [==============================] - 205s 1s/step - loss: 3.9205 - accuracy: 0.7404
Epoch 8/10
163/163 [==============================] - 341s 2s/step - loss: 3.9205 - accuracy: 0.7402
Epoch 9/10
163/163 [==============================] - 226s 1s/step - lo

In [ ]:
model.evaluate(train)

 97/163 [================>.............] - ETA: 35s - loss: 10.3384 - accuracy: 0.7419